# Домашняя работа

Данные берем отызывы за лето
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи 
1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip3 install xlrd

In [5]:
df = pd.read_excel('../homework7/отзывы за лето.xls')

In [6]:
df.head(15)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [7]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [8]:
df.shape

(20659, 3)

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2
import re

In [10]:
morph = pymorphy2.MorphAnalyzer()
stopwords_list = stopwords.words('russian')
punct = set(punctuation)

def tokenize_and_preprocess(text):
    if type(text) is not str:
        return []
    # txt = "".join(c for c in text if c not in punct)
    txt = text
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    words = word_tokenize(txt)
    lemmas = [morph.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords_list and w.isalpha()]

tokenize_and_preprocess('Съешь еще этих мягких французских булок, да выпей же чаю!')

['съесть', 'ещё', 'мягкий', 'французский', 'булка', 'выпить', 'чай']

In [11]:
df['Prepared_Content'] = df['Content'].apply(tokenize_and_preprocess)
df.head(5)

,Rating,Content,Date,Prepared_Content
0,5,It just works!,2017-08-14,"[it, just, works]"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,"[целое, удобноной, приложение, минус, хотеть, ..."
2,5,Отлично все,2017-08-14,"[отлично, всё]"
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,"[стать, зависать, работа, антивирус, далёкий, ..."
4,5,"Очень удобно, работает быстро.",2017-08-14,"[очень, удобно, работать, быстро]"


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_train, df_val = train_test_split(df, test_size=0.25, shuffle=True)

In [14]:
corpus_tokens = []
for i in range(len(df_train)):
    corpus_tokens += df_train["Prepared_Content"].iloc[i]

In [15]:
num_classes = 5
max_words = 5000
max_len = 40

__Создание словаря__

In [16]:
from nltk.probability import FreqDist
dist = FreqDist(corpus_tokens)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [17]:
tokens_filtered_top[:10]

['приложение',
 'всё',
 'очень',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'спасибо',
 'хороший',
 'нравиться']

In [18]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [20]:
def text_to_sequence(tokens, maxlen):
    result = []
    for word in tokens:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen-len(result))
    return padding + result[-maxlen:]

In [21]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Prepared_Content"]], dtype=np.int32)
X_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Prepared_Content"]], dtype=np.int32)

In [22]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

In [23]:
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(df_train['Rating'])
y_val = label_encoder.fit_transform(df_val['Rating'])

In [24]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [25]:
epochs = 20
batch_size = 512
print_batch_n = 100

In [35]:
def compile_train_classifier(model, epochs=20):
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model.fit(X_train, y_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=1,
                     validation_data=(X_val, y_val))

## Рекуррентная сеть

Так как у меня LSTM почему-то не работает на GPU (долго вычисляется), буду обучать только одну эпоху

In [36]:
model_rnn = keras.models.Sequential([
    keras.layers.Embedding(
        input_dim=max_words, output_dim=128,
        input_length=max_len, mask_zero=True),
    keras.layers.LSTM(80, recurrent_dropout=0.2),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Activation("relu"),
    keras.layers.Dense(num_classes),
    keras.layers.Activation('softmax'),
])
compile_train_classifier(model_rnn, epochs=1)

2022-08-31 20:38:50.272601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 1.3992 - accuracy: 0.6819

2022-08-31 20:40:46.470959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 120s 4s/step - loss: 1.3992 - accuracy: 0.6819 - val_loss: 0.9883 - val_accuracy: 0.7113


## Сверточная сеть

In [37]:
model_conv = keras.models.Sequential([
    keras.layers.Embedding(
        input_dim=max_words, output_dim=128,
        input_length=max_len, mask_zero=True),
    keras.layers.Conv1D(128, 7),
    keras.layers.Activation("relu"),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(25),
    keras.layers.Activation("relu"),
    keras.layers.Dense(num_classes),
    keras.layers.Activation('softmax'),
])
compile_train_classifier(model_conv)

Epoch 1/20


2022-08-31 20:41:12.845774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 5s 134ms/step - loss: 1.0653 - accuracy: 0.7045 - val_loss: 0.8411 - val_accuracy: 0.7195
Epoch 2/20


2022-08-31 20:41:17.091223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 3s 108ms/step - loss: 0.7494 - accuracy: 0.7504 - val_loss: 0.6510 - val_accuracy: 0.7741
Epoch 3/20
31/31 [==============================] - 3s 105ms/step - loss: 0.6271 - accuracy: 0.7815 - val_loss: 0.6170 - val_accuracy: 0.7835
Epoch 4/20
31/31 [==============================] - 4s 119ms/step - loss: 0.5698 - accuracy: 0.8000 - val_loss: 0.6106 - val_accuracy: 0.7868
Epoch 5/20
31/31 [==============================] - 4s 120ms/step - loss: 0.5201 - accuracy: 0.8192 - val_loss: 0.6076 - val_accuracy: 0.7905
Epoch 6/20
31/31 [==============================] - 3s 106ms/step - loss: 0.4732 - accuracy: 0.8385 - val_loss: 0.6165 - val_accuracy: 0.7886
Epoch 7/20
31/31 [==============================] - 3s 108ms/step - loss: 0.4275 - accuracy: 0.8595 - val_loss: 0.6382 - val_accuracy: 0.7833
Epoch 8/20
31/31 [==============================] - 4s 122ms/step - loss: 0.3828 - accuracy: 0.8767 - val_loss: 0.6608 - val_accuracy: 0.7839
Epoch 9/20
31/31 

## LSTM + CNN

In [45]:
model_rnn_cnn = keras.models.Sequential([
    keras.layers.Embedding(
        input_dim=max_words, output_dim=128,
        input_length=max_len, mask_zero=True),
    keras.layers.LSTM(80, recurrent_dropout=0.2),
    keras.layers.Reshape((80, 1)),
    keras.layers.Conv1D(128, 7),
    keras.layers.Activation("relu"),
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Activation("relu"),
    keras.layers.Dense(num_classes),
    keras.layers.Activation('softmax'),
])
compile_train_classifier(model_rnn_cnn, epochs=1)

2022-08-31 20:51:30.182550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 1.4322 - accuracy: 0.6837

2022-08-31 20:53:40.879703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 135s 4s/step - loss: 1.4322 - accuracy: 0.6837 - val_loss: 1.0970 - val_accuracy: 0.7121


## Тестирование

In [38]:
def repeat_str(text, n):
    ret = ''
    for i in range(n):
        ret += text
    return ret


def predict(text):
    rating = np.argmax(
        model_conv.predict(
            np.array([
                text_to_sequence(tokenize_and_preprocess(text), max_len)
            ]), verbose=0
        )
    ) + 1
    return rating


def predict_print(text):
    rating = predict(text)
    print(repeat_str('⭐', rating) + ' ' + text)


predict_print('Говно тупое Удаляю!')
predict_print('Говно.')
predict_print('Отличное приложение! Все супер!')
predict_print('Ну нормально, но у меня холодьник от антивируса сломался')

2022-08-31 20:42:20.954620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


⭐ Говно тупое Удаляю!
⭐⭐ Говно.
⭐⭐⭐⭐⭐ Отличное приложение! Все супер!
⭐⭐⭐⭐ Ну нормально, но у меня холодьник от антивируса сломался
